In [1]:
import torch
from PIL import Image
import os
import torchvision.models as models

In [2]:
NO_PATH = "./cloudstreet/no/"
YES_PATH = "./cloudstreet/yes/"
TRAIN_NO_PATH = "./dataset/train/0/"
TRAIN_YES_PATH = "./dataset/train/1/"
TEST_NO_PATH = "./dataset/test/0/"
TEST_YES_PATH = "./dataset/test/1/"
train_no_num = 300
train_yes_num = 400
test_no_num = 200
test_yes_num = 200

In [3]:
lst_no = os.listdir(NO_PATH)
lst_yes = os.listdir(YES_PATH)

In [4]:
import random
datalst_no = random.sample(lst_no, k=(train_no_num+test_no_num))
datalst_yes = random.sample(lst_yes, k=(train_yes_num+test_yes_num))

In [5]:
for i in os.listdir(TRAIN_NO_PATH):
    try:
        os.remove(TRAIN_NO_PATH+i)
    except OSError as e:
        print("Error: %s : %s" % (TRAIN_NO_PATH+i, e.strerror))
        
for i in os.listdir(TRAIN_YES_PATH):
    try:
        os.remove(TRAIN_YES_PATH+i)
    except OSError as e:
        print("Error: %s : %s" % (TRAIN_YES_PATH+i, e.strerror))
        
for i in os.listdir(TEST_NO_PATH):
    try:
        os.remove(TEST_NO_PATH+i)
    except OSError as e:
        print("Error: %s : %s" % (TEST_NO_PATH+i, e.strerror))
        
for i in os.listdir(TEST_YES_PATH):
    try:
        os.remove(TEST_YES_PATH+i)
    except OSError as e:
        print("Error: %s : %s" % (TEST_YES_PATH+i, e.strerror))


In [1]:
for i, n in enumerate(datalst_no):
    img = Image.open(NO_PATH+n).resize((800,800))
    if i < train_no_num:
        img.save(TRAIN_NO_PATH+n)
    else:
        img.save(TEST_NO_PATH+n)
    
for i, n in enumerate(datalst_yes):
    img = Image.open(YES_PATH+n).resize((800,800))
    if i < train_yes_num:
        img.save(TRAIN_YES_PATH+n)
    else:
        img.save(TEST_YES_PATH+n)

NameError: name 'datalst_no' is not defined

In [3]:
print(len(os.listdir(TEST_NO_PATH)), len(os.listdir(TEST_YES_PATH)), len(os.listdir(TRAIN_NO_PATH)), len(os.listdir(TRAIN_YES_PATH)))

200 200 300 400


In [2]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import csv
from model.alexnet import ConvNet

In [3]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 20
num_classes = 2
batch_size = 64
learning_rate = 0.005

# MNIST dataset

train_dataset = torchvision.datasets.ImageFolder(root='./dataset/train/',
                                           transform=transforms.ToTensor())

test_dataset = torchvision.datasets.ImageFolder(root='./dataset/test/',
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)


In [4]:
len(train_loader)

11

In [12]:
# Convolutional neural network (two convolutional layers)
# class ConvNet(nn.Module):
#     def __init__(self, num_classes=10):
#         super(ConvNet, self).__init__()
#         self.layer1 = nn.Sequential(
#             nn.Conv2d(3, 16, kernel_size=6, stride=2, padding=3),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2)
#         )
#         self.layer2 = nn.Sequential(
#             nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=4),
#             nn.BatchNorm2d(32),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2)
#         )
#         self.layer3 = nn.Sequential(
#             nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
#             nn.BatchNorm2d(64),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2)
#         )
#         self.fc1 = nn.Linear(51*51*64, 64)
#         self.fc2 = nn.Linear(64, 2)
#         self.fc3 = nn.Sigmoid()
#         self.relu = nn.ReLU()
#         self.dropout = nn.Dropout(0.1)

#     def forward(self, x):
#         out = self.layer1(x)
#         out = self.layer2(out)
#         out = self.layer3(out)
#         #print(out.size())
#         out = out.reshape(out.size(0), -1)
#         out = self.relu(self.fc1(out))
#         out = self.dropout(out)
#         out = self.fc2(out)
#         out = self.fc3(out)
#         return out



In [10]:

if 'loss.csv' in os.listdir('./'):
    os.remove('loss.csv')

# model = ConvNet(num_classes).to(device)
# model.load_state_dict(torch.load('model.ckpt'))
model = models.inception_v3(num_classes=2)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    loss_rate = 0
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs, _ = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
        loss_rate = loss.item()
    validate_rate = validate(model, test_loader)
    with open(r'loss.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow([loss_rate, validate_rate])
    
    #torch.save(model.state_dict(), 'model.ckpt')

# Test the model

model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs, dummy = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the {} test images: {} %'.format(test_no_num+test_yes_num, 100 * correct / total))

# Save the model checkpoint
# torch.save(model.state_dict(), 'model_twolayer.ckpt')

AttributeError: 'InceptionOutputs' object has no attribute 'log_softmax'

In [12]:
model = ConvNet(num_classes)
model.load_state_dict(torch.load('model.ckpt'))
def validate(model, test_loader):
    model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs, _ = model(images)
            _, predicted = torch.max(outputs.data, 1)
            #print(predicted)
            print("=============")
            #print(labels)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Test Accuracy of the model on the {} test images: {} %'.format(test_no_num+test_yes_num,100 * correct / total))
        return 100*(correct/total)